In [1]:
from operator import add
import pandas as pd
import itertools
from common import Data, Playoffs

In [2]:
df = pd.read_csv("data/2024.csv")
df["season"] = "2024"
teams = list(df["home"].unique())
Data.get_nwsl_standings(df)

,team,wins,draws,losses,goals_for,goals_against,goals_diff,points
1,ORL,17,6,1,43.0,15.0,28.0,57
2,WAS,16,2,6,48.0,28.0,20.0,50
3,NJY,15,5,4,34.0,19.0,15.0,50
4,KC,14,7,3,50.0,29.0,21.0,49
5,NC,12,3,9,34.0,26.0,8.0,39
6,CHI,10,2,12,30.0,33.0,-3.0,32
7,POR,9,4,11,34.0,34.0,0.0,31
8,BFC,9,1,14,27.0,39.0,-12.0,28
9,LOU,6,7,11,31.0,36.0,-5.0,25
10,UTA,7,3,14,20.0,35.0,-15.0,24


In [3]:
g = Playoffs.calc_games_matrix(df, teams)
p0 = Playoffs.calc_initial_points(df, teams)
Playoffs.calculate_lowest_finish("ORL", teams, g, p0)

4

In [4]:
g = Playoffs.calc_games_matrix(df, teams)
p0 = Playoffs.calc_initial_points(df, teams)
Playoffs.calculate_lowest_finish("WAS", teams, g, p0)

5

In [5]:
g = Playoffs.calc_games_matrix(df, teams)
p0 = Playoffs.calc_initial_points(df, teams)
Playoffs.calculate_lowest_finish("KC", teams, g, p0)

5

In [6]:
g = Playoffs.calc_games_matrix(df, teams)
p0 = Playoffs.calc_initial_points(df, teams)
Playoffs.calculate_highest_finish("HOU", teams, g, p0)

6

In [4]:
lookforward = 7
start = min(df.index[df["home_score"].isna()])
outcomes = list([(0, 0), (1, 0), (0, 1)])

In [5]:
options = []
for i in range(lookforward):
    for outcome in outcomes:
        options.append((i, outcome))


# Summarize the scenarios into something meaningful
def format_result(i, outcome):
    match = df.iloc[start + i]
    return f"{match['home']} {outcome[0]} - {match['away']} {outcome[1]}"


def summarize(df_results):
    scenarios = df_results.copy()

    # Check if all values are true
    if sum(scenarios["clinched"]) == len(scenarios.index):
        print("Clinched independent of results")
        return

    # Check single game criteria
    for (g1, o1) in options:
        subset = scenarios[scenarios[g1] == o1]
        if sum(subset["clinched"]) == len(subset.index):
            print("Clinched with", format_result(g1, o1))
            scenarios = scenarios[~(scenarios[g1] == o1)]

    # Check two game criteria
    for ((g1, o1), (g2, o2)) in itertools.combinations(options, 2):
        if g1 == g2:
            continue
        subset = scenarios[(scenarios[g1] == o1) & (scenarios[g2] == o2)]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print("Clinched with", format_result(g1, o1), "and", format_result(g2, o2))
            scenarios = scenarios[~((scenarios[g1] == o1) & (scenarios[g2] == o2))]

    # Check three game criteria
    for ((g1, o1), (g2, o2), (g3, o3)) in itertools.combinations(options, 3):
        if len(set([g1, g2, g3])) < 3:
            continue
        subset = scenarios[
            (scenarios[g1] == o1) & (scenarios[g2] == o2) & (scenarios[g3] == o3)
        ]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print(
                "Clinched with",
                format_result(g1, o1),
                "and",
                format_result(g2, o2),
                "and",
                format_result(g3, o3),
            )
            scenarios = scenarios[
                ~((scenarios[g1] == o1) & (scenarios[g2] == o2) & (scenarios[g3] == o3))
            ]

    # Check four game criteria
    for ((g1, o1), (g2, o2), (g3, o3), (g4, o4)) in itertools.combinations(options, 4):
        if len(set([g1, g2, g3, g4])) < 4:
            continue
        subset = scenarios[
            (scenarios[g1] == o1)
            & (scenarios[g2] == o2)
            & (scenarios[g3] == o3)
            & (scenarios[g4] == o4)
        ]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print(
                "Clinched with",
                format_result(g1, o1),
                "and",
                format_result(g2, o2),
                "and",
                format_result(g3, o3),
                "and",
                format_result(g4, o4),
            )
            scenarios = scenarios[
                ~(
                    (scenarios[g1] == o1)
                    & (scenarios[g2] == o2)
                    & (scenarios[g3] == o3)
                    & (scenarios[g4] == o4)
                )
            ]

    # Check five game criteria
    for ((g1, o1), (g2, o2), (g3, o3), (g4, o4), (g5, o5)) in itertools.combinations(
        options, 5
    ):
        if len(set([g1, g2, g3, g4, g5])) < 5:
            continue
        subset = scenarios[
            (scenarios[g1] == o1)
            & (scenarios[g2] == o2)
            & (scenarios[g3] == o3)
            & (scenarios[g4] == o4)
            & (scenarios[g5] == o5)
        ]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print(
                "Clinched with",
                format_result(g1, o1),
                "and",
                format_result(g2, o2),
                "and",
                format_result(g3, o3),
                "and",
                format_result(g4, o4),
                "and",
                format_result(g5, o5),
            )
            scenarios = scenarios[
                ~(
                    (scenarios[g1] == o1)
                    & (scenarios[g2] == o2)
                    & (scenarios[g3] == o3)
                    & (scenarios[g4] == o4)
                    & (scenarios[g5] == o5)
                )
            ]

    # Check six game criteria
    for (
        (g1, o1),
        (g2, o2),
        (g3, o3),
        (g4, o4),
        (g5, o5),
        (g6, o6),
    ) in itertools.combinations(options, 6):
        if len(set([g1, g2, g3, g4, g5, g6])) < 6:
            continue
        subset = scenarios[
            (scenarios[g1] == o1)
            & (scenarios[g2] == o2)
            & (scenarios[g3] == o3)
            & (scenarios[g4] == o4)
            & (scenarios[g5] == o5)
            & (scenarios[g6] == o6)
        ]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print(
                "Clinched with",
                format_result(g1, o1),
                "and",
                format_result(g2, o2),
                "and",
                format_result(g3, o3),
                "and",
                format_result(g4, o4),
                "and",
                format_result(g5, o5),
                "and",
                format_result(g6, o6),
            )
            scenarios = scenarios[
                ~(
                    (scenarios[g1] == o1)
                    & (scenarios[g2] == o2)
                    & (scenarios[g3] == o3)
                    & (scenarios[g4] == o4)
                    & (scenarios[g5] == o5)
                    & (scenarios[g6] == o6)
                )
            ]

In [6]:
# Calculate all scenarios for the following week and whether a specific team
# has clinched a playoff spot
def evaluate_scenarios(team, threshold):
    df_copy = df.copy()

    df_copy.loc[start : start + (lookforward - 1), "home_score"] = [0] * lookforward
    df_copy.loc[start : start + (lookforward - 1), "away_score"] = [0] * lookforward

    p0 = Playoffs.calc_initial_points(df, teams)
    g = Playoffs.calc_games_matrix(df_copy, teams)

    subset = df_copy.loc[start : start + (lookforward - 1)]

    def calculate_clinch(row):
        subset["home_score"] = list(row.str[0])
        subset["away_score"] = list(row.str[1])
        return (
            Playoffs.calculate_lowest_finish(
                team, teams, g, list(map(add, p0, Playoffs.calc_initial_points(subset, teams, True)))
            )
            <= threshold
        )

    scenarios = pd.DataFrame(itertools.product(outcomes, repeat=lookforward))
    scenarios["clinched"] = scenarios.apply(calculate_clinch, axis=1)
    return scenarios

In [7]:
# Calculate all scenarios for the following week and whether a specific team
# has been eliminated from the playoffs
def evaluate_elimination_scenarios(team, threshold):
    df_copy = df.copy()

    df_copy.loc[start : start + (lookforward - 1), "home_score"] = [0] * lookforward
    df_copy.loc[start : start + (lookforward - 1), "away_score"] = [0] * lookforward

    p0 = Playoffs.calc_initial_points(df, teams)
    g = Playoffs.calc_games_matrix(df_copy, teams)

    subset = df_copy.loc[start : start + (lookforward - 1)]

    def calculate_clinch(row):
        subset["home_score"] = list(row.str[0])
        subset["away_score"] = list(row.str[1])
        return (
            Playoffs.calculate_highest_finish(
                team, teams, g, list(map(add, p0, Playoffs.calc_initial_points(subset, teams, True)))
            )
            > threshold
        )

    scenarios = pd.DataFrame(itertools.product(outcomes, repeat=lookforward))
    scenarios["clinched"] = scenarios.apply(calculate_clinch, axis=1)
    return scenarios

## Clinching playoffs

Already clinched: ORL, WAS, KC, NJY, NC, CHI

In [9]:
summarize(evaluate_scenarios("POR", 8))

Clinched with LOU 0 - POR 0
Clinched with LOU 0 - POR 1


In [10]:
summarize(evaluate_scenarios("BFC", 8))

Clinched with LOU 0 - POR 0 and BFC 1 - NC 0
Clinched with LOU 0 - POR 1 and BFC 1 - NC 0
Clinched with LOU 0 - POR 1 and BFC 0 - NC 0 and LA 0 - UTA 0


In [11]:
summarize(evaluate_scenarios("LOU", 8))

## Clinching hosting

Already clinched: ORL, WAS, KC, NJY

In [10]:
summarize(evaluate_scenarios("WAS", 4))

Clinched with NC 0 - LA 0
Clinched with NC 0 - LA 1
Clinched with WAS 0 - LOU 0
Clinched with WAS 1 - LOU 0


In [11]:
summarize(evaluate_scenarios("KC", 4))

Clinched with NC 0 - LA 0
Clinched with NC 0 - LA 1
Clinched with BFC 0 - KC 1


In [12]:
summarize(evaluate_scenarios("NJY", 4))

Clinched with CHI 0 - NJY 0
Clinched with CHI 0 - NJY 1
Clinched with NC 0 - LA 0
Clinched with NC 0 - LA 1


## Clinching shield

Already clinched: ORL

In [13]:
summarize(evaluate_scenarios("ORL", 1))

Clinched independent of results


## Eliminated from playoffs
Already eliminated: HOU, RGN

In [12]:
summarize(evaluate_elimination_scenarios("UTA", 8))

Clinched with BFC 1 - NC 0
Clinched with LA 1 - UTA 0
Clinched with BFC 0 - NC 0 and LA 0 - UTA 0


In [13]:
summarize(evaluate_elimination_scenarios("LA", 8))

Clinched with BFC 1 - NC 0
Clinched with LA 0 - UTA 0
Clinched with LA 0 - UTA 1


In [14]:
summarize(evaluate_elimination_scenarios("SD", 8))

Clinched with KC 0 - SD 0
Clinched with KC 1 - SD 0
Clinched with BFC 0 - NC 0
Clinched with BFC 1 - NC 0


In [15]:
summarize(evaluate_elimination_scenarios("LOU", 8))

Clinched with LOU 0 - POR 0 and BFC 1 - NC 0
Clinched with LOU 0 - POR 1 and BFC 0 - NC 0
Clinched with LOU 0 - POR 1 and BFC 1 - NC 0


In [16]:
summarize(evaluate_elimination_scenarios("BFC", 8))

## Eliminated from hosting
Already eliminated: HOU, UTA, RGN, LA, SD, LOU, BAY, POR, CHI

In [18]:
summarize(evaluate_elimination_scenarios("NC", 4))

Clinched with NC 0 - LA 0
Clinched with NC 0 - LA 1
Clinched with CHI 0 - NJY 0 and BFC 0 - KC 1 and WAS 0 - LOU 0
Clinched with CHI 0 - NJY 0 and BFC 0 - KC 1 and WAS 1 - LOU 0
Clinched with CHI 0 - NJY 1 and BFC 0 - KC 1 and WAS 0 - LOU 0
Clinched with CHI 0 - NJY 1 and BFC 0 - KC 1 and WAS 1 - LOU 0


## Eliminated from shield
Already eliminated: everyone but ORL

In [22]:
summarize(evaluate_elimination_scenarios("KC", 1))

Clinched with LOU 0 - KC 0
Clinched with LOU 1 - KC 0
Clinched with ORL 1 - WAS 0


In [23]:
summarize(evaluate_elimination_scenarios("NJY", 1))

Clinched with NJY 0 - BFC 1
Clinched with ORL 1 - WAS 0
Clinched with NJY 0 - BFC 0 and ORL 0 - WAS 0


In [24]:
summarize(evaluate_elimination_scenarios("WAS", 1))

Clinched with ORL 1 - WAS 0


In [20]:
g = Playoffs.calc_games_matrix(df, teams)
p0 = Playoffs.calc_initial_points(df, teams)

for team in teams:
    print(
        team,
        "can finish:",
        Playoffs.calculate_highest_finish(team, teams, g, p0),
        "-",
        Playoffs.calculate_lowest_finish(team, teams, g, p0),
    )

KC can finish: 2 - 4
LOU can finish: 7 - 13
UTA can finish: 8 - 14
NC can finish: 5 - 5
RGN can finish: 9 - 14
LA can finish: 8 - 14
ORL can finish: 1 - 1
CHI can finish: 6 - 8
WAS can finish: 2 - 4
SD can finish: 8 - 14
HOU can finish: 9 - 14
POR can finish: 6 - 9
BFC can finish: 6 - 11
NJY can finish: 2 - 4
